This notebook is part of Andreu's (esdandreu@gmail.com) Master Thesis work at
Keio University.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AcousticOdometry/AO/blob/main/notebooks/plot_features.ipynb)


# Setup
This section will take care of installing the necessary packages as well as
configuring some environment variables.

In [1]:
COLAB_RUNTIME = False
GITHUB_TOKEN = None

## Colab
Assess wether the notebook is being executed in [Google
Colab](https://colab.research.google.com/) and if so, set up the software
needed in Colab runtime.

In [2]:
%%capture
try:
    import colab
    COLAB_RUNTIME = True
    # Check CMake >= 3.21
    v_str, *_ = !cmake --version
    if 'command not found' in v_str:
        major, minor = (0, 0)
    else:
        major, minor, _ = (int(x) for x in v_str.split(' ')[-1].split('.'))
    if major < 3 or minor < 21:
        # https://cmake.org/download/
        %cd /tmp
        !wget https://github.com/Kitware/CMake/releases/download/v3.22.3/cmake-3.22.3-linux-x86_64.sh
        !sudo mkdir /opt/cmake
        !sudo sh ./cmake-3.22.3-linux-x86_64.sh --prefix=/opt/cmake \
            --skip-license
        %cd /content
        !update-alternatives --install /usr/local/bin/cmake cmake \
            /opt/cmake/bin/cmake 20 --force
except ImportError:
    pass

## AO
Setup Acoustic Odometry python package. If this notebook is being executed in
[Colab](#colab), the package will be installed from Github. Because of this, a
Github [personal access
token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)
will be asked during the installation.

If the notebook is not running on Colab and the package is not already
installed, installation instructions will be prompted.

In [3]:
if COLAB_RUNTIME:
    from getpass import getpass
    if not GITHUB_TOKEN:
        GITHUB_TOKEN = getpass(
            "Personal access token\n"
            r"https://docs.github.com/en/authentication/keeping-your-account-"
            r"and-data-secure/creating-a-personal-access-token"+"\n"
            )
    %pip install git+https://$GITHUB_TOKEN@github.com/AcousticOdometry/AO
    try:
        import ao
    except ImportError:
        GITHUB_TOKEN = None
        raise
else:
    try:
        import ao
    except ImportError:
        raise ImportError(
            "Acoustic Odometry python extension is not installed. Check "
            r"https://github.com/AcousticOdometry/AO#readme"
            " for detailed instructions."
            )

## Other packages


In [4]:
import ao
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

c:\Users\esdan\Desktop\AO\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

## Model definition

In [5]:
class TransformerEncoder(layers.Layer):

    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
            )
        self.ffn = keras.Sequential([
            layers.Dense(feed_forward_dim, activation="relu"),
            layers.Dense(embed_dim),
            ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [6]:
class Transformer(keras.Model):
    pass

## Load data

Data folder should be provided as an environment variable. It can be written in
an `.env` file in the root of the project.

In [7]:
if COLAB_RUNTIME:
    # TODO Mount drive and find VAO_Primitive-Experiment
    # TODO Otherwise prompt to add shortcut to drive from link
    # https://drive.google.com/drive/folders/1I6dq8gJpsrD3C14-WQKD4IFOnkiVuvFX
    raise NotImplementedError()
else:
    DATA_FOLDER = ao.dataset.utils.get_data_folder(env='PRIMITIVE_EXPERIMENT')

In [8]:
data, naming = ao.dataset.utils.list_data(DATA_FOLDER)
print(next(iter(data.items())))
# TODO create a tensor with parameters
# TODO map parameters tensor into motion tensor

(WindowsPath('C:/Users/esdan/OneDrive - keio.jp/Thesis/Primitive Experiment/w_10;load_0;contact_no'), {'w': 10, 'load': 0, 'contact': False})


c:\Users\esdan\Desktop\AO\venv\lib\site-packages\ao\dataset\utils.py:124: UserWarning: Could not parse VAO_2022-03-08_17-11-21 from C:\Users\esdan\OneDrive - keio.jp\Thesis\Primitive Experiment\VAO_2022-03-08_17-11-21, item VAO_2022-03-08_17-11-21 should be composed by one `key` and one `value` separated by a unique underscore but 2 `_` were found.
  warn(str(e))
c:\Users\esdan\Desktop\AO\venv\lib\site-packages\ao\dataset\utils.py:124: UserWarning: Could not parse VAO_2022-03-08_17-12-37 from C:\Users\esdan\OneDrive - keio.jp\Thesis\Primitive Experiment\VAO_2022-03-08_17-12-37, item VAO_2022-03-08_17-12-37 should be composed by one `key` and one `value` separated by a unique underscore but 2 `_` were found.
  warn(str(e))


In [9]:
# Create a tensor with file names
files = tf.data.Dataset.from_tensor_slices([
    str(p / 'audio0crop.wav') for p in data.keys()
    ])

for f in tfds.as_numpy(files.take(2)):
    print(f)

b'C:\\Users\\esdan\\OneDrive - keio.jp\\Thesis\\Primitive Experiment\\w_10;load_0;contact_no\\audio0crop.wav'
b'C:\\Users\\esdan\\OneDrive - keio.jp\\Thesis\\Primitive Experiment\\w_10;load_0;contact_yes\\audio0crop.wav'


In [10]:
# Map folder tensor into an audio features tensor

FRAME_SAMPLES = 1024
NUM_FEATURES = 64

audio_sample, samplerate = ao.io.wave_read(list(data.keys())[0] / 'audio0.wav')
_extract = ao.extractor.GammatoneFilterbank(
    FRAME_SAMPLES, NUM_FEATURES, samplerate
    )


@tf.function
def extract(frame):
    tf.print(frame.numpy())
    return _extract(frame)

@tf.function
def audio_features(filename):
    audio, _ = tf.audio.decode_wav(tf.io.read_file(filename), 1)
    audio_samples = tf.shape(audio)[0]
    tf.print(audio_samples)
    cropped_size = audio_samples - audio_samples % FRAME_SAMPLES
    audio_ = tf.reshape(
        audio[:cropped_size], [cropped_size // FRAME_SAMPLES, FRAME_SAMPLES]
        )
    audio_ = tf.map_fn(extract, audio_)
    # print(audio_frames)
    # audio_frames = tf.RaggedTensor.from_tensor(audio_frames)
    # print(audio_frames)
    # audio_features = tf.TensorArray(audio.dtype, size=0, dynamic_size=True)
    # for i, frame in enumerate(tf.split(audio, [FRAME_SAMPLES], axis=0)):
    #     audio_features = audio_features.write(i, extract(frame))
    # return audio_features.stack()
    return audio_


audio_ds = files.map(audio_features)

for a in tfds.as_numpy(audio_ds.take(1)):
    print(a)

AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
FRAME_SAMPLES = 1024
NUM_FEATURES = 64

audio_sample, samplerate = ao.io.wave_read(list(data.keys())[0] / 'audio0.wav')
extract = ao.extractor.GammatoneFilterbank(
    FRAME_SAMPLES, NUM_FEATURES, samplerate
    )

data_list = [(str(folder), parameters) for folder, parameters in data.items()]


def generate_audio_dataset(
        data_list,
        frame_samples: int = FRAME_SAMPLES,
        num_features: int = NUM_FEATURES
    ):
    # TODO initialize extract in generator
    for folder, parameters in data_list:
        audio_path = folder / 'audio0crop.wav'
        audio, _samplerate = ao.io.wave_read(audio_path)
        assert _samplerate == samplerate, "Samplerate mismatch"
        num_frames = tf.math.ceil(audio.size / frame_samples)
        vx = parameters['contact'] * parameters['w'] * 0.10  # 10 cm radius
        motion = (vx, 0)  # No angular rotation
        # Pad the data to be a multiple of the frame size
        audio = np.append(
            audio, np.zeros(num_frames * frame_samples - audio.size)
            )
        for frame_num in range(num_frames):
            frame = audio[frame_num * frame_samples:(frame_num + 1) *
                          frame_samples]
            frame_features = extract(frame)
            yield (frame_features, motion)


audio_ds = tf.data.Dataset.from_generator(
    generate_audio_dataset,
    output_signature=(
        tf.TensorSpec(shape=None, dtype=tf.float32),
        tf.TensorSpec(shape=None, dtype=tf.float32)
        ),
    args=(data_list, )
    )
print(audio_ds.take(1))

ValueError: Can't convert Python sequence with mixed types to Tensor.